In [1]:
from langchain_community.llms import Ollama
llm = Ollama(model="llama3")

response = llm.invoke("What is llm")
print(response)


LLM stands for Large Language Model. It's a type of artificial intelligence (AI) model that has been trained on a massive dataset of text and has learned to generate human-like language outputs.

Large Language Models are designed to process and understand vast amounts of text data, allowing them to perform tasks such as:

1. **Language translation**: LLMs can translate text from one language to another.
2. **Text summarization**: They can summarize long pieces of text into shorter summaries.
3. **Question answering**: LLMs can answer questions based on the content they've been trained on.
4. **Creative writing**: Some LLMs are capable of generating creative writing, such as stories or poetry.
5. **Conversational dialogue**: They can engage in natural-sounding conversations with humans.

LLMs use a combination of techniques, including:

1. **Self-supervised learning**: The model is trained on large amounts of text data without explicit supervision.
2. **Attention mechanisms**: LLMs use

In [2]:
from langchain_community.document_loaders import DirectoryLoader
from langchain_community.document_loaders.csv_loader import CSVLoader
from langchain.document_loaders import PyPDFDirectoryLoader

loader = PyPDFDirectoryLoader("pdfs")
docs = loader.load()
docs

[Document(page_content='Research Journal of Chemical Sciences______________ _______________________________Vol. 1 (2) May (2011 ) \n                                    Res.J.Chem.Sci  \n \n135 \n Review Paper \nTargeted drug delivery system: A Review \nGupta Manish and Sharma Vimukta \nBM College of Pharmaceutical Education and Research , Indore, INDIA \n \nAvailable online at: www.isca.in  \n(Received 17 th   March 2011, revised 04 th  May 2011, accepted 05 th  May 2011) \n \nAbstract \nTargeted drug delivery is a method of delivering me dication to a patient in a manner that increases th e \nconcentration of the medication in some parts of th e body relative to others. Targeted drug delivery \nseeks to concentrate the medication in the tissues of interest while reducing the relative concentrati on of \nthe medication in the remaining tissues. This impro ves efficacy of the while reducing side effects. It  is \nvery difficult for a drug molecule to reach its des tination in the comple

In [3]:
print(docs[1])


page_content='Research Journal of Chemical Sciences______________ _______________________________Vol. 1 (2) May (2011 ) \n                                    Res.J.Chem.Sci  \n \n136 \n strain, impurities), the presence of an interface \nbetween different semiconductor materials (e.g. in \ncore-shell nanocrystal systems), the presence of th e \nsemiconductor surface (e.g. semiconductor \nnanocrystal), or a combination of these. Quantum \ndots are particularly significant for optical \napplications due to their theoretically high quantu m \nyield. The ability to tune the size of quantum dots  is \nadvantageous for many applications and it is one of  \nthe most promising candidates for use in solid-stat e \nquantum computation and diagnosis , drug delivery, \nTissue engineering, catalysis, filtration and also \ntextiles technologies. \n \n                         \n \n \n \nTransdermal Approach: Transdermal drug delivery \nsystem is topically administered medicaments in the  \nform of pa

In [4]:
from langchain_community.embeddings import OllamaEmbeddings
embeddings = OllamaEmbeddings(model='llama3')

In [5]:
from langchain_community.vectorstores import FAISS
from langchain_text_splitters import RecursiveCharacterTextSplitter
import pickle

text_splitter = RecursiveCharacterTextSplitter()
documents = text_splitter.split_documents(docs)
vector = FAISS.from_documents(documents, embeddings)


In [ ]:

with open("vector", "wb") as f:
    pickle.dump(vector, f)
# django-admin startproject myapp

In [6]:
from langchain.chains import create_history_aware_retriever
from langchain_core.prompts import ChatPromptTemplate

from langchain_core.prompts import MessagesPlaceholder

# First we need a prompt that we can pass into an LLM to generate this search query
retriever = vector.as_retriever()
prompt = ChatPromptTemplate.from_messages([
    MessagesPlaceholder(variable_name="chat_history"),
    ("user", "{input}"),
    ("user", "Given the above conversation, generate a search query to look up to get information relevant to the conversation")
])
document_chain = create_history_aware_retriever(llm, retriever, prompt)

In [7]:
from langchain_core.prompts import ChatPromptTemplate

from langchain.chains import create_history_aware_retriever
from langchain_core.prompts import MessagesPlaceholder
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain

retriever = vector.as_retriever()

# First we need a prompt that we can pass into an LLM to generate this search query

prompt = ChatPromptTemplate.from_template(
    """ 
    
     A set of documents related to pharmaceuticals and drug interactions is provided.
     Your task is to extract information from these documents to accurately respond
     to inquiries about drug effects, interactions, regulations, and other related topics.
     Initially, search the document to find precise answers.
     If the document does not contain the information or if the query requires additional context,
     use your pre-trained knowledge to provide the most accurate response possible.
     Note that the documents might contain complex medical terminology and data that requires careful interpretation.
     When answering, ensure the response is simplified for general understanding while maintaining medical accuracy.
     If a query pertains to adverse effects or safety guidelines and the document is inconclusive, prioritize safety in your response.

<context>
{context}
</context>

Question: {input}"""
)


document_chain = create_stuff_documents_chain(llm, prompt)
# retriever_chain = create_history_aware_retriever(llm, retriever, prompt)

In [8]:
from langchain.chains import create_retrieval_chain
import pickle

with open("vector", "rb") as f:
    vector = pickle.load(f)

retriever = vector.as_retriever()
retrieval_chain = create_retrieval_chain(retriever, document_chain)

In [11]:
response = retrieval_chain.invoke({"input": """" why has liposomes generated great interest?"""})
print(response['answer'])

# LangSmith offers several features that can help with testing:...

Liposomes have generated great interest because they offer a versatile and effective way to deliver therapeutic agents, such as drugs, genes, and vaccines, to specific sites in the body. The key advantages of liposomes are:

1. **Targeted delivery**: Liposomes can be designed to target specific cells, tissues, or organs, allowing for more efficient and effective treatment.
2. **Improved bioavailability**: Liposomes can protect therapeutic agents from degradation, increasing their bioavailability and reducing side effects.
3. **Increased efficacy**: By delivering therapeutic agents directly to the site of action, liposomes can increase the efficacy of treatments.
4. **Reduced toxicity**: Liposomes can reduce systemic exposure to therapeutic agents, minimizing toxicity and improving patient safety.

The interest in liposomes has been fueled by their potential applications in various fields, including:

1. Cancer treatment: Liposomes have shown promise in delivering cancer therapies, such